In [0]:
# GraphFrames jar needs to be installed,select jar file corresponding with version of spark and scala!
# https://spark-packages.org/package/graphframes/graphframes
import graphframes as gf

spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism) #shuffle partitions for faster processing of graph algorithms

help(gf) # GraphFrame = 2 dataframes: vertices and edges

Help on package graphframes:

NAME
    graphframes

PACKAGE CONTENTS
    examples (package)
    graphframe
    lib (package)
    tests

CLASSES
    builtins.object
        graphframes.graphframe.GraphFrame
    
    class GraphFrame(builtins.object)
     |  GraphFrame(v, e)
     |  
     |  Represents a graph with vertices and edges stored as DataFrames.
     |  
     |  :param v:  :class:`DataFrame` holding vertex information.
     |             Must contain a column named "id" that stores unique
     |             vertex IDs.
     |  :param e:  :class:`DataFrame` holding edge information.
     |             Must contain two columns "src" and "dst" storing source
     |             vertex IDs and destination vertex IDs of edges, respectively.
     |  
     |  >>> localVertices = [(1,"A"), (2,"B"), (3, "C")]
     |  >>> localEdges = [(1,2,"love"), (2,1,"hate"), (2,3,"follow")]
     |  >>> v = sqlContext.createDataFrame(localVertices, ["id", "name"])
     |  >>> e = sqlContext.createData

In [0]:
df = spark.read.option("multiline", True).json('dbfs:/user/dblpv13/dblpv13.0.json.gz')

#### Vertices/Nodes
*  Must contain the **id** column

In [0]:
import pyspark.sql.functions as F

**Author's Node**

In [0]:
#Can create these from our already cleaned dataframes
author_node = (df
                .select(F.col("_id").alias("id"))
                .union(df
                      .select(F.explode(F.col("authors._id").alias("id"))))
                .distinct()
)

display(author_node.limit(10))

id
53e99784b7602d9701f3e3f5
53e99784b7602d9701f3e162
53e99784b7602d9701f3e4f4
53e99784b7602d9701f3e161
53e99784b7602d9701f3e922
53e99784b7602d9701f3eaf2
53e99784b7602d9701f3e165
53e99784b7602d9701f3e133
53e99784b7602d9701f3e151
53e99784b7602d9701f3e15d


In [0]:
# must contain src and dst columns
authors_edge = (df
               .select(F.explode(F.col("authors._id")).alias("src")
                      ,F.col("_id").alias("dst")
                      ,F.col("*"))
                .distinct()
)

display(authors_edge.limit(10))

src,dst,_id,abstract,authors,doi,fos,isbn,issn,issue,keywords,lang,n_citation,page_end,page_start,pdf,references,title,url,venue,volume,year
53f45728dabfaec09f209538,53e99784b7602d9701f3e133,53e99784b7602d9701f3e133,"Drought is the first place in all the natural disasters in the world. It is especially serious in North China Plain. In this paper, different soil water content control levels at winter wheat growth stages are performed on Gucheng Ecological-Meteorological Integrated Observation Experiment Station of CAMS, China. Some canopy parameters, including growth conditions, dry weight, physiological parameters and hyperspectral reflectance, are measured from erecting stage to milk stage for winter wheat in 2009. The relationship between canopy parameters and soil relative moisture, canopy water content and water indices of winter wheat are established. The results show that some parameters, such as SPAD and dry weight of leaves, decrease with the increasing of soil relative moisture, while other parameters, including dry weight of caudexes, above ground dry weight, height, photosynthesis rate, intercellular CO 2 concentration, stomatal conductance and transpiration rate, increase corresponding to the soil relative moisture. Obvious linear relationship between stomatal conductance and transpiration rate is established with 45 samples, which R2 reaches to 0.6152. Finally, the fitting equations between canopy water content and water indices are regressed with b5, b6 and b7 of MODIS bands. The equations are best with b7 and worst with b5. So the fitting equations with b7 can be used to inverse the canopy water content of winter wheat using MODIS or other remote sensing images with similar bands range to MODIS in Hebei Province. © 2011 IEEE.","List(List(53f45728dabfaec09f209538, null, null, null, Peijuan Wang, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e59457, null, null, null, Jiahua Zhang, null, null, null, null, null, null, null, null, null, null), List(53f38438dabfae4b34a08928, null, null, null, Donghui Xie, null, null, null, null, null, null, null, null, null, null), List(5601754345cedb3395e5945a, null, null, null, Yanyan Xu, null, null, null, null, null, null, null, null, null, null), List(53f43d25dabfaeecd6995149, null, null, null, Yun Xu, null, null, null, null, null, null, null, null, null, null))",10.1109/IGARSS.2011.6049503,"List(Agronomy, Moisture, Hydrology, Environmental science, Dry weight, Water content, Stomatal conductance, Transpiration, Irrigation, Soil water, Canopy)",,,null,"List(canopy parameters, canopy spectrum, different soil water content control, winter wheat, irrigation, hydrology, radiometry, moisture, indexes, vegetation, indexation, dry weight, soil moisture, water content, indexing terms, spectrum, natural disaster)",en,0,1933,1930,null,null,The relationship between canopy parameters and spectrum of winter wheat under different irrigations in Hebei Province.,List(http://dx.doi.org/10.1109/IGARSS.2011.6049503),"List(53a7297d20f7420be8bd4ae7, null, null, International Geoscience and Remote Sensing Symposium, null, null, null, IGARSS, null, null, null, null, 0)",null,2011
53f45ee9dabfaee43ecda842,53e99784b7602d9701f3e15d,53e99784b7602d9701f3e15d,"As process variations become a significant problem in deep sub-micron technology, a shift from deterministic static timing analysis to statistical static timing analysis for high-performance circuit designs could reduce the excessive conservatism that is built into current timing design methods. We address the timing yield problem for sequential circuits and propose a statistical approach to handle it. We consider the spatial and path reconvergence correlations between path delays, set-up time and hold time constraints, and clock skew due to process variations. We propose a method to get the timing yield based on the delay distributions of register-to-register paths in the circuit On average, the timing yield result

In [0]:
# Let's create our first GraphFrame

authors_graph = gf.GraphFrame(author_node, authors_edge)

display(authors_graph)

GraphFrame(v:[id: string], e:[src: string, dst: string ... 20 more fields])